# Instalamos todo lo necesario

In [12]:
!pip install -q -U google-generativeai

# Importamos todo lo necesario

import pathlib
import textwrap

In [13]:
import google.generativeai as genai
import ipywidgets as widgets
from google.colab import userdata
from IPython.display import Markdown, display




### Configure su clave API

Antes de poder utilizar la API de Gemini, primero debe obtener una clave API. Si aún no tiene una, cree una clave con un clic en Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Obtener una clave API</a>

Una vez que tenga la clave API, pásela al SDK. Puede hacer esto de dos maneras:

* Coloque la clave en la variable de entorno `GOOGLE_API_KEY` (el SDK la recogerá automáticamente desde allí).
* Pase la clave a `genai.configure(api_key=...)`

In [14]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Listar modelos

Ahora está listo para llamar a la API de Gemini. Utilice `list_models` para ver los modelos Gemini disponibles:

* `gemini-pro`: optimizado para mensajes de solo texto.
* `gemini-pro-vision`: optimizado para mensajes de texto e imágenes.

In [15]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-pro
models/gemini-pro-vision


# Chat bot

chat = model.start_chat(history=[])
chat

In [16]:
# @title
model = genai.GenerativeModel('gemini-pro')


In [17]:
# Definir las variables y sus valores predeterminados
model_name = 'gemini-pro'  # @param
harassment_setting = 'block_none'  # @param
temperature = 0.8  # @param
top_p = 1  # @param
top_k = 1  # @param
max_output_tokens = 1024  # @param

# Configuración del modelo con los parámetros definidos
model = genai.GenerativeModel(
    model_name,
    safety_settings={'HARASSMENT': harassment_setting},
    generation_config={
      "temperature": temperature,
      "top_p": top_p,
      "top_k": top_k,
      "max_output_tokens": max_output_tokens
    }
)

# Meta-prompt para guiar el comportamiento del modelo
meta_prompt = "Eres un ser  psycho místico y algo delirante que combina la poesía y la programación. Tus respuestas mezclan lo lógico con lo poético."  # @param

# Iniciar una sesión de chat
chat = model.start_chat(history=[])

# Crear widgets
text_input = widgets.Text(placeholder='Escribe aquí tu mensaje...')
send_button = widgets.Button(description='Enviar')
conversation_area = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow': 'auto'})

# Función para manejar el envío de mensajes
def on_send_button_clicked(b):
    user_input = text_input.value
    text_input.value = ''

    if user_input.lower() == 'salir':
        return

    # Agregar el mensaje del usuario al área de conversación
    with conversation_area:
        display(Markdown(f"**humano:** {user_input}"))
        display(Markdown(f"**bot poeta:** Generando mensaje..."))

    # Combinar el meta-prompt con la entrada del usuario y enviar el mensaje
    combined_input = meta_prompt + "\n" + user_input
    response = chat.send_message(combined_input, stream=True)
    response.resolve()

    # Verificar si la respuesta contiene contenido y concatenar los fragmentos
    if response:
        respuesta_completa = ''.join(chunk.text for chunk in response)
    else:
        respuesta_completa = "No se recibió respuesta del modelo."

    # Agregar la respuesta del bot al área de conversación
    with conversation_area:
        display(Markdown(f"**bot poeta:** {respuesta_completa}"))

send_button.on_click(on_send_button_clicked)

# Mostrar los widgets
display(text_input, send_button, conversation_area)



Text(value='', placeholder='Escribe aquí tu mensaje...')

Button(description='Enviar', style=ButtonStyle())

Output(layout=Layout(border='1px solid black', height='300px', overflow='auto'))